In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import pandas as pd
from urllib.request import urlopen
from datetime import datetime as dt
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)
from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv('PM2.5_2019 (EPA).csv',sep = ',')
df_new = pd.read_csv('counties.csv',sep = ',')
df_new['val'] = 0
df_new.dropna(subset=['FIPS #'],inplace = True)
#df = df.loc[:,['COUNTY_CODE','STATE','Site Name','SITE_LATITUDE','SITE_LONGITUDE','COUNTY','Daily Mean PM2.5 Concentration','CBSA_NAME']].groupby(['COUNTY_CODE','CBSA_NAME','STATE','Site Name','SITE_LATITUDE','SITE_LONGITUDE','COUNTY']).mean()
df = df.loc[:,['COUNTY_CODE','Daily Mean PM2.5 Concentration','COUNTY']].groupby(['COUNTY_CODE','COUNTY']).mean()
df.reset_index(inplace = True)
for index, row in df_new.iterrows():
    if(len(df[df['COUNTY'] == df_new.loc[index,' County Name '].strip()]) > 0):
        b = (df[df['COUNTY'] == df_new.loc[index,' County Name '].strip()])
        #print(b.iloc[0]['Daily Mean PM2.5 Concentration'])
        df_new.loc[index,'val'] = b.iloc[0]['Daily Mean PM2.5 Concentration']
        #print('hello')
    
df_new.reset_index(inplace=True)    
df_new['FIPS #'] = df_new['FIPS #']+48000
df_new['FIPS #'] = df_new['FIPS #'].astype(int).astype(str)
#48000 + (county_number * 2) - 1

In [3]:
df_new['variable'] = 'PM2.5' #key to identify different variables
df_new.head()

,index,County Name,FIPS #,County #,Metropolitan Statistical Area (MSA),val,variable
0,0,Anderson,48001,1.0,--,0.0,PM2.5
1,1,Andrews,48003,2.0,--,0.0,PM2.5
2,2,Angelina,48005,3.0,--,0.0,PM2.5
3,3,Aransas,48007,4.0,Corpus Christi,0.0,PM2.5
4,4,Archer,48009,5.0,Wichita Falls,0.0,PM2.5


In [4]:
# separate station variables and county variables
# this list is not complete
variable_type = {
    'XCO2':'county',
    'XCH4':'county',
    'NO2':'station',
    'PM2.5':'county',
    'Dissolved Oxygen':'station',
    'Orthophosphate':'station',
    'COVID Cases':'county'
}
# unit for each variable
unitmap = {
    'XCO2':'ppm of ',
    'XCH4':'ppm of ',
    'NO2':'ppm of ',
    'PM2.5':'ppm of ', 
    'Dissolved Oxygen':'blah unit of ',
    'Orthophosphate':'blah unit of ',
    'COVID Cases':''
}

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    html.H1("Texas Environmental Data and COVID19", style={'text-align': 'center'}),
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(1995, 8, 5),
        max_date_allowed=dt(2017, 9, 19),
        initial_visible_month=dt(2017, 8, 5),
        date=str(dt(2017, 8, 25, 23, 59, 59))),
    dcc.Dropdown(id="slct_var",
                 options=[
                     {"label": "Greenhouse Gas - XCO2", "value": "XCO2"},
                     {"label": "Greenhouse Gas - XCH4", "value": "XCH4"},
                     {"label": "Air Quality - NO2", "value": "NO2"},
                     {"label": "Air Quality - PM2.5", "value": "PM2.5"},
                     {"label": "Water Quality - Dissolved Oxygen", "value": "Dissolved Oxygen"},
                     {"label": "Water Quality - Orthophosphate", "value": "Orthophosphate"},
                     {"label": "COVID Cases", "value": "COVID Cases"}],
                 multi=False,
                 value="XCO2",
                 style={'width': "40%"}
                 ),
    dcc.Checklist(id = 'slct_watermap',
        options=[
            {'label': 'Aquifers', 'value': 'Aquifers'},
            {'label': 'Water Shed', 'value': 'Watershed'} # we can ignore river basin
        ],
        value=['Aquifers','Watershed']),
    html.Div(id='output_container', children=[]),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure={}),
    ])

#==================== Update map based on selected variables and watershed ====================#
@app.callback([Output('output_container','children'),Output('map','figure')],
    [Input('slct_var','value'),Input('slct_watermap','value')])

def update_fig(option_var, option_water):
    # subset dataframe with the selected variable
    dff = df_new[df_new.variable == option_var]
    
    mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw' 
    # output1: text indicating variable selection. This is for demo, we can remove it for the final product
    container = "The variable chosen by user was: {}. Display {} on the map".format(option_var,option_water)
    
    # output2: map
    # if the variable is by county
    if variable_type[option_var] == 'county':
        trace= go.Choroplethmapbox(geojson=counties,
                                   locations=dff['FIPS #'],
                                   z=dff['val'],
                                   colorscale='ice',
                                   colorbar_thickness=20,
                                   text = dff[' County Name '],
                                   marker_line_color='white',customdata = dff['FIPS #'], hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                                    '<br> <b>Val </b>: %{z}<br>'+  '<extra></extra>'
                                )
        fig= go.Figure(data = trace)                          
        fig.update_layout(title_text= 'County Average of {} from [date] to [date]'.format(option_var),
                          title_x=0.5, width = 700, height=700,
                          mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                         accesstoken= mapboxt,style = 'basic',
                                         zoom=4.5,
                                       ))

    # if the variable is by station
    # replace the code below with Ben's code
    if variable_type[option_var] == 'station':
        trace= go.Choroplethmapbox(geojson=counties,
                                   locations=dff['FIPS #'],
                                   z=dff['val'],
                                   colorscale='ice',
                                   colorbar_thickness=20,
                                   text = dff[' County Name '],
                                   marker_line_color='white',customdata = dff['FIPS #'], hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                                    '<br> <b>Val </b>: %{z}<br>'+  '<extra></extra>'
                                )
        fig= go.Figure(data = trace)                          
        fig.update_layout(title_text= 'Station Average of {} from [date] to [date]'.format(option_var),
                          title_x=0.5, width = 700, height=700,
                          mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                         accesstoken= mapboxt,style = 'basic',
                                         zoom=4.5,
                                       ))
    return container, fig

#==================== Update hover text ====================#
@app.callback(Output('text-content','children'),
    [Input('map','hoverData'), Input('slct_var','value')])

def update_text(hoverData,option_var):
    dff = df_new[df_new.variable == option_var]
    
    # if data for this day is missing
    if dff.shape[0]==0:
        return html.H3("no data for this variable")
    
    if(hoverData == None):
        return html.H3("Hover over a county or station")
    s = dff[dff['FIPS #'] == hoverData['points'][0]['customdata']]
    # Blank county has blank /units/ of /parameter/
    return html.H3(
        '{} County has {} {}{}'.format(
            s.iloc[0][' County Name '],
            s.iloc[0]['val'],
            unitmap[option_var],
            option_var))


app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


/Users/joannezhou/anaconda3/lib/python3.7/site-packages/dash/resources.py:68: UserWarning: You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources
  ).format(s["external_url"])
